<a href="https://colab.research.google.com/github/bok-h22/TIL-202303/blob/master/OPEN_API_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#서울시 공공 API

In [ ]:
import json
import requests


url="http://openapi.seoul.go.kr:8088/key/json/tpssSubwayPassenger/1/423/"
response = requests.get(url)
data = response.json()
data

In [ ]:
data = response.json()
subway = []

for i in data['tpssSubwayPassenger']['row']:
    data = {}
    data['날짜'] = i['CRTR_DT']
    data['행정동 ID'] = i['ADMDONG_ID']
    data['지하철 승객 수'] = i['SBWY_PSGR_CNT']

    subway.append(data)

subway

In [ ]:
subway_df = pd.DataFrame(subway)
subway_df

In [ ]:
import json
import requests

url="http://openapi.seoul.go.kr:8088/key/json/tpssEmdBus/1/423/"
response = requests.get(url)
response.status_code

In [ ]:
data = response.json()
bus = []

for i in data['tpssEmdBus']['row']:
    data = {}
    data['행정동 ID'] = i['ADMDONG_ID']
    data['버스 승객 수'] = i['BUS_PSGR_CNT']

    bus.append(data)

bus

In [ ]:
bus_df = pd.DataFrame(bus)
bus_df.head()

bus_list=list(bus_df['행정동 ID'])
type(bus_list)
bus_list

행정동 데이터

In [ ]:
url='http://openapi.seoul.go.kr:8088/key/json/districtEmd/1/424/'
response = requests.get(url)
response.status_code

#데이터 가져오기
data = response.json()
dong = []

for j in data['districtEmd']['row']:
    data = {}
    data['행정동 ID'] = j['ADMDONG_ID']
    data['시'] = j['SIDO_NM']
    data['구'] = j['ATDRC_NM']
    data['동'] = j['ADMDONG_NM']
    dong.append(data)

dong_df = pd.DataFrame(dong)
dong_df = dong_df[dong_df['시']=='서울']
dong_df

In [ ]:
url='http://openapi.seoul.go.kr:8088/key/json/districtEmd/1/424/'
response = requests.get(url)
response.status_code

#데이터 가져오기
data = response.json()
data['districtEmd']['row']

#카카오 맵 API

In [ ]:
#담꾹 좌표 불러오기
import pandas as pd
x_y = pd.read_excel('/content/밀키트 위도,경도.xlsx').values.tolist()
x_y[1][3]

37.5531476

#인근 밀키트 매장 갯수 구하기

In [ ]:
import requests
import json
import pandas as pd

url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
kakao_key = 'key'

인근_밀키트_매장_수= []


for i in range(len(x_y)):

    stop = True

    mil_cnt = 0
    page = 1
    page_size = 45

    data = {}

    mil_name = x_y[i][1]
    mil_x = x_y[i][3]
    mil_y = x_y[i][2]

    while stop:
        params = {'query' : '밀키트',
            'x' : mil_x,
            'y' : mil_y,
            'page' : page,
            'page_size': page_size,
            'radius' : 500}
        header = {"Authorization":f"KakaoAK {kakao_key}"}

        milkit = requests.get(url = url,
                                params = params,
                                headers = header).json()

        mil_cnt += len(milkit['documents'])

        if milkit['meta']['is_end'] == False:
            stop = True
            page += 1
        else:
            stop = False

    #데이터 넣기
    data['매장명'] = mil_name
    data['인근 밀키트 매장 수'] = mil_cnt

    인근_밀키트_매장_수.append(data)

인근_밀키트_매장_수 = pd.DataFrame(인근_밀키트_매장_수)
인근_밀키트_매장_수



In [ ]:
인근_밀키트_매장_수

,매장명,인근 밀키트 매장 수
0,집밥뚝딱 먹골역점,45
1,집밥뚝딱 약수점,45
2,집밥뚝딱 신길뉴타운점,45
3,집밥뚝딱 롯데캐슬 파크엘점,45
4,집밥뚝딱 헬리오시티점,45
...,...,...
180,담꾹 헬리오시티점,45
181,담꾹 홍은점,45
182,담꾹 홍제역점,45
183,담꾹 화곡본동점,45


In [ ]:
#output
인근_밀키트_매장_수
인근_밀키트_매장_수.to_csv("./인근 밀키트 매장 수 (1km).csv", encoding="utf-8-sig")

#인근 대형마트 매장 갯수 구하기

In [ ]:
import requests
import json
import pandas as pd

url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

인근_대형마트_매장_수= []

for i in range(len(x_y)):

    data = {}

    mil_name = x_y[i][1]
    mil_x = x_y[i][3]
    mil_y = x_y[i][2]

    params = {'query' : '대형마트',
                'x' : mil_x,
                'y' : mil_y,
                'radius' : 1000}

    kakao_key = '2dc142a121aa04c7f2cf477c8daa9074'

    milkit = requests.get(url = url,
                            params = params,
                            headers = {"Authorization":f"KakaoAK {kakao_key}"}).json()["documents"]
    mart_cnt = len(milkit)

    #데이터 넣기
    data['매장명'] = mil_name
    data['인근 대형마트 매장 수'] = mart_cnt

    인근_대형마트_매장_수.append(data)

인근_대형마트_매장_수 = pd.DataFrame(인근_대형마트_매장_수)
인근_대형마트_매장_수

,매장명,인근 대형마트 매장 수
0,집밥뚝딱 먹골역점,15
1,집밥뚝딱 약수점,15
2,집밥뚝딱 신길뉴타운점,15
3,집밥뚝딱 롯데캐슬 파크엘점,15
4,집밥뚝딱 헬리오시티점,15
...,...,...
180,담꾹 헬리오시티점,15
181,담꾹 홍은점,15
182,담꾹 홍제역점,15
183,담꾹 화곡본동점,15


In [ ]:
#output
인근_대형마트_매장_수.to_csv("./인근 대형마트 매장 수 (1km).csv", encoding="utf-8-sig")

#인근 편의점 갯수

인근 편의점 개수

In [ ]:
import requests
import json
import pandas as pd

url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
kakao_key = 'key'

인근_편의점_매장_수= []


for i in range(len(x_y)):

    stop = True

    conv_cnt = 0
    page = 1
    page_size = 45

    data = {}

    mil_name = x_y[i][1]
    mil_x = x_y[i][3]
    mil_y = x_y[i][2]

    while stop:
        params = {'query' : '편의점',
            'x' : mil_x,
            'y' : mil_y,
            'page' : page,
            'page_size': page_size,
            'radius' : 500}
        header = {"Authorization":f"KakaoAK {kakao_key}"}

        milkit = requests.get(url = url,
                                params = params,
                                headers = header).json()

        conv_cnt += len(milkit['documents'])

        if milkit['meta']['is_end'] == False:
            stop = True
            page += 1
        else:
            stop = False

    #데이터 넣기
    data['매장명'] = mil_name
    data['인근 편의점 매장 수'] = conv_cnt

    인근_편의점_매장_수.append(data)

인근_편의점_매장_수 = pd.DataFrame(인근_편의점_매장_수)




In [ ]:
#output
인근_편의점_매장_수.to_csv("./인근 편의점 매장 수 (500m).csv", encoding="utf-8-sig")

인근 아파트 세대수

#좌표로 행정동 추출

In [ ]:
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
kakao_key = 'key'


담꾹_행정동 = []

for i in range(len(x_y)):

    mil_name = x_y[i][1]
    mil_x = x_y[i][3]
    mil_y = x_y[i][2]

    params = {'x' : mil_x,
              'y' : mil_y}

    hjd = requests.get(url = url,
                        params = params,
                        headers = {"Authorization":f"KakaoAK {kakao_key}"}).json()['documents'][1]

    data = {}

    #데이터 입력
    data['매장명'] = mil_name
    data['행정동 code'] = hjd["code"]
    data['행정동 주소'] = hjd["address_name"]
    data['시'] = hjd["region_1depth_name"]
    data['구'] = hjd["region_2depth_name"]
    data['동'] = hjd["region_3depth_name"]
    data['etc'] = hjd["region_4depth_name"]
    data['경도'] = mil_y
    data['위도'] = mil_x

    담꾹_행정동.append(data)

담꾹_행정동 = pd.DataFrame(담꾹_행정동)

KeyError: ignored

#최종 데이터 병합

In [ ]:
담꾹_지도API_최종 = pd.merge(담꾹_행정동, 인근_밀키트_매장_수, on = "매장명", how = "inner")
담꾹_지도API_최종 = pd.merge(담꾹_지도API_최종, 인근_편의점_매장_수, on = "매장명", how = "inner")
담꾹_지도API_최종 = pd.merge(담꾹_지도API_최종, 인근_대형마트_매장_수, on = "매장명", how = "inner")
담꾹_지도API_최종

In [ ]:
#output
담꾹_지도API_최종.to_csv("/Users/asd/Desktop/담꾹 지도API 최종.csv", encoding="utf-8-sig")

In [ ]:
import pandas as pd
import requests
아파트코드_최종 = pd.read_csv('/Users/asd/Desktop/아파트코드_최종_없음삭제.csv').values.tolist()
아파트세대수 = []
for bjd_code in range(30):
    아파트코드 = 아파트코드_최종[bjd_code][2]
    url = f'https://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo?serviceKey=key%2B6k2O1ojfc%2FxvyMMw%3D%3D&_type=json&kaptCode={아파트코드}'
    response = requests.get(url)
    data = {}
    if 아파트코드 == '없음':
        print('아파트 없음')
        pass
    else:
        ex1 = response.json()['response']['body']['item']
        data['아파트명'] = ex1['kaptName']
        data['아파트 code'] = ex1['kaptCode']
        data['법정동코드'] = ex1['bjdCode']
        data['아파트 세대수'] = ex1['kaptdaCnt']
        print(data)
        아파트세대수.append(data)